In [2]:
from aerolib import *
from xfoillib import *
from exceptions import *
from inner_iteration import *

m = inner_iteration()

rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory



  XFOIL Version 6.99
  Copyright (C) 2000   Mark Drela, Harold Youngren

  This software comes with ABSOLUTELY NO WARRANTY,
    subject to the GNU General Public License.

  Caveat computor

 File  xfoil.def  not found

   QUIT    Exit program

  .OPER    Direct operating point(s)
  .MDES    Complex mapping design routine
  .QDES    Surface speed design routine
  .GDES    Geometry design routine

   SAVE f  Write airfoil to labeled coordinate file
   PSAV f  Write airfoil to plain coordinate file
   ISAV f  Write airfoil to ISES coordinate file
   MSAV f  Write airfoil to MSES coordinate file
   REVE    Reverse written-airfoil node ordering
   DELI i  Change written-airfoil file delimiters

   LOAD f  Read buffer airfoil from coordinate file
   NACA i  Set NACA 4,5-digit airfoil and buffer airfoil
   INTE    Set buffer airfoil by interpolating two airfoils
   NORM    Buffer airfoil normalization toggle
   HALF    Halve the number of points in buffer airfoil
   XYCM rr Change CM refere

rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory



 XFOIL   c>   SD   command not recognized.  Type a "?" for list

 XFOIL   c>  
 Sharp trailing edge

 Paneling parameters used...
   Number of panel nodes       160
   Panel bunching parameter    1.000
   TE/LE panel density ratio   0.150
   Refined-area/LE panel density ratio    0.200
   Top    side refined area x/c limits  1.000 1.000
   Bottom side refined area x/c limits  1.000 1.000

 XFOIL   c>  
.OPERi   c>  
 M  =    0.0000
 Re =  0.2854E+06

.OPERv   c>  
.OPERv   c>  
 M  =    0.0000
 Re =  0.2854E+06

.OPERv   c>  
 Xtr/c     =  1.0000    top    side
 Xtr/c     =  1.0000    bottom side
 NcritT    =    9.00   ( 0.070 % turb. level )
 NcritB    =    9.00   ( 0.070 % turb. level )
 Vacc      =  0.0100
 WakeL/c   =   1.000

 Klag  =  5.6000     Uxwt  =    1.00       Kdl =  0.9000
 A     =  6.7000     B     =  0.7500       KCt = 0.01485
 CtiniK=  1.8000     CtiniX=  3.3000

..VPAR   c>  
..VPAR   c>  
.OPERv   c>   Current iteration limit:          20

Enter new iteration limit 

rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory


...

rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory



 Solving BL system ...

 Initializing BL ...
    side            1  ...
 MRCHUE: Inverse mode at  50     Hk =   3.913
 MRCHUE: Inverse mode at  51     Hk =   2.500
    side            2  ...
 MRCHUE: Inverse mode at  10     Hk =   6.497
 MRCHUE: Inverse mode at  11     Hk =   9.610
 MRCHUE: Inverse mode at  12     Hk =   2.500

 Side 1  free  transition at x/c =  0.5485   50
 Side 2  free  transition at x/c =  0.0124   12

   1   rms: 0.1629E+00   max: -.1837E+01   D at   12  2   RLX: 0.272
       a = -1.051      CL =  0.2595
      Cm = -0.0781     CD =  0.00591   =>   CDf =  0.00515    CDp =  0.00075

 MRCHDU: Convergence failed at  12  side 2    Res =  0.3143E+01
 Side 1  free  transition at x/c =  0.5508   50
 Side 2  free  transition at x/c =  0.0143   12

   2   rms: 0.9151E-01   max: -.1180E+01   D at   11  2   RLX: 0.424
       a = -1.027      CL =  0.2607
      Cm = -0.0780     CD =  0.00584   =>   CDf =  0.00516    CDp =  0.00067

 Side 1  free  transition at x/c =  0.5533   

rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory
rm: cannot remove '/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/polar.dat': No such file or directory



 XFOIL   c>   SD   command not recognized.  Type a "?" for list

 XFOIL   c>  
 Sharp trailing edge

 Paneling parameters used...
   Number of panel nodes       160
   Panel bunching parameter    1.000
   TE/LE panel density ratio   0.150
   Refined-area/LE panel density ratio    0.200
   Top    side refined area x/c limits  1.000 1.000
   Bottom side refined area x/c limits  1.000 1.000

 XFOIL   c>  
.OPERi   c>  
 M  =    0.0000
 Re =  0.3771E+07

.OPERv   c>  
.OPERv   c>  
 M  =    0.0000
 Re =  0.3771E+07

.OPERv   c>  
 Xtr/c     =  1.0000    top    side
 Xtr/c     =  1.0000    bottom side
 NcritT    =    9.00   ( 0.070 % turb. level )
 NcritB    =    9.00   ( 0.070 % turb. level )
 Vacc      =  0.0100
 WakeL/c   =   1.000

 Klag  =  5.6000     Uxwt  =    1.00       Kdl =  0.9000
 A     =  6.7000     B     =  0.7500       KCt = 0.01485
 CtiniK=  1.8000     CtiniX=  3.3000

..VPAR   c>  
..VPAR   c>  
.OPERv   c>   Current iteration limit:          20

Enter new iteration limit 

введите 1, чтобы произвести ещё 5 операций, иначе 0:  0


In [1]:
from params import *
from geometry import *
from mass_dependencies import weigh

def log_mass():
    file = open("mass_dependencies.py", "r")
    mass_list_started = False
    for line in file.readlines():
        if line[:12] == "def weigh():":
            mass_list_started = True
            continue
        if mass_list_started:
            if line.isspace():
                continue
            line = line[4:-1]
            if line[0] == "#":
                print()
                print(line[1:])
            elif line[0] == "M" and line.split() != ["M", "=", "0"]:
                expression = " ".join(line.split()[2:]) 
                print(expression + " = ", end="" )
                exec("print(" + expression + ")")

    print("="*20)
    print(f"M = {weigh()}")
    file.close()



In [3]:
indicate_missing_param_vals = 0
show_keys_in_logs = 0
weigh("PARAMS.csv", "GEOM.csv", indicate_missing_param_vals, show_keys_in_logs)

dict_items([(1.0, [0.6, 0.013]), (2.0, [0.024, 1.0]), (3.0, [0.005, 1.0]), (4.0, [0.01, 1.0]), (5.0, [0.025, 2.0]), (6.0, [0.058, 1.0]), (7.0, [0.047, 3.0]), (8.0, [0.0068, 0.0]), ('specific_wing', [1.05, 0.3054504693848956, 7.777777777777778]), ('specific_aft', [1.05, 0.22437444358659897]), (13.0, [0.15, 1.0]), (9.0, [0.015, 0.0]), (10.0, [0.006, 4.0]), ('specific_3wire', [0.0095, 4.382036711805679]), (11.0, [0.048, 1.0]), (12.0, [0.0])])
 + l_stab * tube6X5: 0.0078
 + mVTX * nVTX: 0.0318
 + mCrossfire * nCrossfire: 0.0368
 + mRunc_s4 * nRunc_s4: 0.0468
 + mReg * nReg: 0.0968
 + mAT2308 * nAT2308: 0.1548
 + mLiIon18650 * nLiIon18650: 0.2958
 + mLiIon21700 * nLiIon21700: 0.2958
 + specific_wing * wing_area * AR: 2.790312166643314
 + specific_aft * Vtail_area: 3.025905332409243
 + mfus * nfus: 3.175905332409243
 + mMG90 * nMG90: 3.175905332409243
 + mES9051 * nES9051: 3.199905332409243
 + specific_3wire * wire_length: 3.2415346811713968
 + mPixhawk * nPixhawk: 3.289534681171397
 + nPito

3.289534681171397

In [3]:
XFoil_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
work_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
#XFoil_command_CL(16, 7, 0.15, 0.8, 1.5e-5, 0, 9, 1000, XFoil_path, work_path, "SD7037")

In [4]:
'''
Модуль расчёта геометрических характеристик потребляет на вход: 
dataframe PERFORMANCE с потребными эксплуатационными характеристиками
dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
TOM - значение взлётной массы в начальном приближении.

PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
area_dens_LWPLA
energy_dens_bat
number_servo1, number_servo2
l_stab
'''

# Все единицы в СИ
def inner_iteration():
    performance_file_name = "PERFORMANCE.csv"#input("имя файла performances: ")
    params_file_name = "PARAMS.csv" #input("имя файла params: ")
    settings_file_name = "SETTINGS.csv" #input("имя файла settings: ") 
    tom = 2 #input("нулевое приближение влётной массы: ")
    max_iter = 10 #input("максимальное число итераций: ")
    
    df = pd.read_csv(settings_file_name, header=None)
    tom_eps = float(dict(zip(df[0], df[1]))["tom_eps"])
    
    sculptor = Sculptor(performance_file_name, params_file_name, settings_file_name, tom)
    i = 1
    ask_about_no_value = True
    while True:
        sculptor.calculate_geometry()
        sculptor.write_info()
        pf, gf = sculptor.get_data_to_weigh()

        print(f"итерация {i}")
        new_tom = weigh(pf, gf, ask_about_no_value)
        ask_about_no_value = False
        if abs(tom - new_tom) > tom_eps:
            tom = new_tom
            sculptor.update_m(new_tom)
        else:
            print(f"сошлось на итерации: {i}")
            print("информация сохранена в файл с геометрией")
            return new_tom
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                print("последняя итерация геометрии сохранена")
                break
            
        i+=1
    